In [6]:
import pandas as pd

# 1. Load Data (assuming you have the dataframe 'rfm' from the previous step in memory)
rfm = pd.read_csv('rfm_with_clusters.csv') 
# (Make sure to save the previous step's result if you haven't!)



In [7]:
# 2. Define the Mapping Dictionary (Cluster ID -> Tier Name)
# Based on your specific results:
cluster_mapping = {
    2: 'Platinum',  # High Value + Frequent
    1: 'Gold',      # New / Recent
    0: 'Silver',    # High Value / Old (Churn Risk)
    3: 'Bronze'     # Low Value / Old (Lost)
}

rfm['Tier'] = rfm['Cluster'].map(cluster_mapping)

In [14]:
# 3. The "Brain" Function (Assigning Actions)
def assign_campaign(row):
    # 1. SENTIMENT GUARDRAIL (Safety First)
    if row['Avg_Review_Score'] <= 2:
        return 'Customer Service: Apology & Remediation'
    
    # Define Category Groups for cleaner logic
    tech_cats = ['telefonia', 'informatica_acessorios', 'eletronicos', 'pcs']
    home_cats = ['cama_mesa_banho', 'moveis_decoracao', 'utilidades_domesticas']
    fashion_cats = ['beleza_saude', 'relogios_presentes', 'esporte_lazer']
    
    # 2. PLATINUM STRATEGY (Loyalty)
    if row['Tier'] == 'Platinum':
        if row['Favorite_Category'] in tech_cats:
            return 'VIP Tech: Early Access to New Gadgets'
        elif row['Favorite_Category'] in home_cats:
             return 'VIP Home: Concierge Interior Design'
        else:
            return 'VIP General: Exclusive Concierge Offer'
            
    # 3. GOLD STRATEGY (Retention/Upsell)
    elif row['Tier'] == 'Gold':
        # Split by Category
        if row['Favorite_Category'] in tech_cats:
            return 'Retention: Upgrade your setup (10% Off)'
        elif row['Favorite_Category'] in fashion_cats:
            return 'Retention: New Season Styles (10% Off)'
        else:
            return 'Retention: 10% Off Your Second Order'
        
    # 4. SILVER STRATEGY (Reactivation)
    elif row['Tier'] == 'Silver':
         # Split by Category
        if row['Favorite_Category'] in home_cats:
            return 'Win-Back: New Home Decor Trends'
        else:
            return 'Win-Back: We Miss You (High Value)'
        
    # 5. BRONZE STRATEGY (Liquidation)
    else:
        return 'Standard: Check out our daily deals'

In [15]:
# 4. Apply the Logic
rfm['Marketing_Action'] = rfm.apply(assign_campaign, axis=1)

In [16]:
# 5. Review the Results
print("Strategy Assignment Summary:")
print(rfm['Marketing_Action'].value_counts())

Strategy Assignment Summary:
Marketing_Action
Standard: Check out our daily deals        37069
Win-Back: We Miss You (High Value)         21869
Customer Service: Apology & Remediation    12378
Retention: 10% Off Your Second Order        9165
Win-Back: New Home Decor Trends             4913
Retention: New Season Styles (10% Off)      3743
VIP General: Exclusive Concierge Offer      1663
Retention: Upgrade your setup (10% Off)     1636
VIP Home: Concierge Interior Design          674
VIP Tech: Early Access to New Gadgets        248
Name: count, dtype: int64


In [17]:
# 6. Save Final Action List for HubSpot
rfm.to_csv('final_campaign_list.csv', index=False)
print("\n Saved Actionable List to 'data/processed/final_campaign_list.csv'")


 Saved Actionable List to 'data/processed/final_campaign_list.csv'


In [18]:
# --- Visual Check ---
# Let's see how many "Platinum" users are actually "Angry"
angry_vips = rfm[(rfm['Tier'] == 'Platinum') & (rfm['Marketing_Action'].str.contains('Apology'))]
print(f"\n⚠️ Critical: Found {len(angry_vips)} Angry VIPs who need immediate human intervention.")


⚠️ Critical: Found 216 Angry VIPs who need immediate human intervention.
